<a href="https://colab.research.google.com/github/polcord/SegmentationModel/blob/main/Preparacion_modelamiento_agrupacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# <center>  Preparación y modelamiento de datos para Agrupación </center>

- Nota: Este notebook fue ejecutado en la plataforma Google Colaboratory y se cargaron los datos de forma manual en el entorno de ejecución

## Descripción

- En esta sección utilizaremos procesos de ingeniería de variable para preparar el conjunto de datos antes del entrenamiento de los algoritmos.
- Posteriomente, se ejecutarán los algoritmos de PCA y KMeans para encontrar el índice que recoja el comportamiento y posterior el agrupamiento de los datos.


### Preguntas

1. Categorizar y describir a los clientes en grupos.

**Respuesta:**

- La categoría 0 se asocia a clientes de **alto valor**, porque poseen en promedio 24 consumos, con un monto promedio transaccionado cercano a 5000 y un ingreso promedio aproximado a 1700. Esta combinación de alta frecuencia de consumos y recursos financieros sugiere que pertenecen a un segmento socioeconómico más próspero y, por lo tanto, se asocian con un mayor valor para la empresa.
-  La categoría 2 se asocia a clientes de **medio valor**, porque poseen en promedio 9 consumos, con un monto promedio transaccionado cercano a 1000 y un ingreso promedio aproximado a 1000. Esto sugiere que tienen una capacidad financiera suficiente para respaldar sus transacciones, aunque a un nivel más moderado en comparación con el grupo de alto valor. En términos socioeconómicos, estos clientes podrían situarse en un segmento medio en cuanto a capacidad económica.
-  La categoría 1 se asocia a clientes de **bajo valor**, porque poseen en promedio 2 consumos, con un monto promedio transaccionado cercano a 50 y un ingreso promedio aproximado a 900. Sus montos transaccionados son modestos y aunque tienen un ingreso promedio moderado, su nivel general de actividad transaccional y recursos disponibles los sitúa en un segmento socioeconómico con limitaciones.

**A continuación, encuentra el código que detalla la respuesta a cada una de las preguntas antes expuestas.**

## Importar librerías y verificar versiones

In [ ]:
!pip install pyreadr

In [ ]:
import sys
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib
import pyreadr

In [ ]:
print('** Versiones Actuales | Requeridas **')
print('Python:\nVersion Actual:', sys.version, ' | Requerida >= 3.6')
print('NumPy:\nVersion Actual: {:10} | Requerida >= 1.16.2'.format(np.__version__))
print('Pandas:\nVersion Actual: {:10}| Requerida >= 0.24.2'.format(pd.__version__))
print('Scikit-learn:\nVersion Actual: {:10}| Requerida >= 1.2.1'.format(sk.__version__))
print('Matplotlib:\nVersion Actual: {:10} | Requerida >= 3.0.3'.format(matplotlib.__version__))
print('Seaborn:\nVersion Actual: {:10} |Requerida >= 0.9.0 '.format(sns.__version__))

** Versiones Actuales | Requeridas **
Python:
Version Actual: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]  | Requerida >= 3.6
NumPy:
Version Actual: 1.23.5     | Requerida >= 1.16.2
Pandas:
Version Actual: 1.5.3     | Requerida >= 0.24.2
Scikit-learn:
Version Actual: 1.2.2     | Requerida >= 1.2.1
Matplotlib:
Version Actual: 3.7.1      | Requerida >= 3.0.3
Seaborn:
Version Actual: 0.12.2     |Requerida >= 0.9.0 


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA, FastICA

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

## Carga de los datos

Se creó un dataframe de Python para trabajar con el conjunto de datos estructurados de los clientes.

Con ello, se disponen de 5000 registros en el conjunto de datos.

In [ ]:
# Ruta del directorio en la nube
ruta1 = './FRI.RData'

# Lectura de los datos usando pyreadr (creación de DataFrames)
datos = pyreadr.read_r(ruta1)
datos = datos["FRI"]

print('\n No. filas y columnas', datos.shape)

datos.head()


 No. filas y columnas (5000, 3)


,Frecuencia,Transaccionalidad,Ingresos
0,5.0,121.36,1918.00
1,1.0,0.02,2000.00
2,36.0,6786.87,860.69
3,8.0,62.87,1354.39
4,12.0,1978.01,1586.03


### EDA

Al realizar el perfilamiento de los datos y su análisis exploratorio se identifican las siguientes características en el conjunto de datos:

- No se encontraron valores perdidos en el conjunto de datos
- Solo de disponen de variables continuas (numéricas)
- El campo ingresos presenta una fuerte asimetría a la derecha de su distribución γ1 = 67.83 y un 9% de sus registros poseen el valor de cero.

In [ ]:
datos.describe().round()

,Frecuencia,Transaccionalidad,Ingresos
count,5000.0,5000.0,5000.0
mean,14.0,2578.0,1270.0
std,13.0,6953.0,14852.0
min,1.0,0.0,0.0
25%,4.0,248.0,560.0
50%,11.0,1521.0,849.0
75%,20.0,2462.0,1200.0
max,148.0,190518.0,1036000.0


 Se empleó la transformación de Yeo-Johnson para ajustar los datos y que se asemejen una distribución normal. Lo cual facilita los procesos estadísticos que se realizaron en los siguientes pasos.

In [ ]:
# Transformar datos para reducir sesgo por asimetría
from scipy import stats

datos_trans = datos.copy()
datos_trans['Frecuencia'], _ = stats.yeojohnson(datos_trans['Frecuencia'])
datos_trans['Transaccionalidad'], _ = stats.yeojohnson(datos_trans['Transaccionalidad'])
datos_trans['Ingresos'], _ = stats.yeojohnson(datos_trans['Ingresos'])

datos_trans.describe()

,Frecuencia,Transaccionalidad,Ingresos
count,5000.000000,5000.000000,5000.000000
mean,2.898762,15.667154,19.593960
std,1.305069,8.677783,8.541779
min,0.731795,0.009961,0.000000
25%,1.828229,10.672391,18.152358
50%,3.031960,18.038041,20.921117
75%,3.891636,20.534807,23.487773
max,7.564331,60.125320,187.623479


In [ ]:
# Seleccionar features
X = datos_trans.loc[:, :]

A continuación, se muestran los campos que se emplearán para el proceso de preparación de datos antes del modelamiento.

In [ ]:
X.columns

Index(['Frecuencia', 'Transaccionalidad', 'Ingresos'], dtype='object')

### Ingeniería de variable para variables numéricas y categóricas

In [ ]:
# Selección de los nombres de las variables numéricas
X_num = X.select_dtypes(include=np.number).columns
print(X_num)

Index(['Frecuencia', 'Transaccionalidad', 'Ingresos'], dtype='object')


### Pipeline - Ingeniería de variable

Se distingue el tratamiento o ingeniería de variables para los campos númericos.

**Variables numéricas**

Para cada campo se procede a realizar la imputación de los valores perdidos por la mediana, medida de tendencia central que no es influenciable por valores atípicos/extremos. Esto en caso de que en algún momento se presenten valores perdidos aunque no es el caso en este conjunto de datos.

Posterior se ejecutó el proceso de estandarización necesario antes del aplicar el PCA.

Finalmente, empleamos el algoritmo de Análisis de Componentes Principales (PCA) obteniendo los componentes principales.

In [ ]:
SEED = 42

In [ ]:
# Creación de un Pipeline para el preprocesamiento de las variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ("scaler", StandardScaler()),
    ('pca', PCA(n_components=1, random_state=SEED))])

Se observó que el primer componente principal recoge el 61% de la varianza explicada.

Donde los pesos de cada predictor para la primera componente principal fue:
- Frecuencia (67%)
- Transaccionalidad (68%)
- Ingresos (26%)


In [ ]:
numeric_transformer.fit(X)
print('Ratio Varianza Explicada:', numeric_transformer.named_steps['pca'].explained_variance_ratio_.cumsum(), '\n')

for i in range(len(X.columns)):
  print(X.columns[i], numeric_transformer.named_steps['pca'].components_.flatten()[i])

Ratio Varianza Explicada: [0.61450772] 

Frecuencia 0.6769362632257488
Transaccionalidad 0.6861473706548085
Ingresos 0.2663814582013029


In [ ]:
# Delimitar el preprocesamiento de variables
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X_num)], verbose_feature_names_out=True)

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler()),
                                                 ('pca',
                                                  PCA(n_components=1,
                                                      random_state=42))]),
                                 Index(['Frecuencia', 'Transaccionalidad', 'Ingresos'], dtype='object'))])

## Modelamiento

### Modelo de Kmeans

Delimitamos que el algoritmo de KMeans posea tres clústers para identificar a los clientes en tres categorías: alto, medio y bajo

In [ ]:
k_means_model = KMeans(n_clusters=3, init='k-means++', n_init=10, random_state=SEED)

In [ ]:
# Construir el pipeline completo con KMeans
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('kmeans', k_means_model)
])

# Ajustar el pipeline a los datos
pipeline.fit(X)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=1,
                                                                       random_state=42))]),
                                                  Index(['Frecuencia', 'Transaccionalidad', 'Ingresos'], dtype='object'))])),
                ('kmeans', KMeans(n_clusters=3, n_init=10, random_state=42))])

In [ ]:
# Obtener las etiquetas de grupo asignadas por KMeans
labels = pipeline.named_steps['kmeans'].labels_

# Asignar las etiquetas a los registros
df = datos.copy()
df['label'] = labels

## Categorizar a cada grupo

Se observa que existe una similar proporción entre las tres categorías obtenidas a partir del emplear el algoritmo de KMeans.

- La categoría 0 se asocia a clientes de alto valor, porque poseen en promedio una frecuencia, transaccionalidad e ingresos más alto que el resto de segmentos.
- La categoría 2 se asocia a clientes de medio valor, porque poseen en promedio un valor intermedio en frecuencia y transaccionalidad, sin embargo son ligeramente más alto sus ingresos que el segmento de bajo valor.
- La categoría 1 se asocia a clientes de bajo valor, porque poseen en promedio una frecuencia, transaccionalidad e ingresos menor que el resto de segmentos.

In [ ]:
df.label.value_counts(normalize=True)

0    0.4304
2    0.3502
1    0.2194
Name: label, dtype: float64

In [ ]:
df.groupby('label').agg(['mean']).round(2)

,Frecuencia,Transaccionalidad,Ingresos
,mean,mean,mean
label,,,
0,24.06,5077.68,1699.52
1,1.92,49.07,880.46
2,8.87,1091.16,987.29
